# EcoMap.BR - Análise Exploratória Econômica

**Projeto:** Análise de Complexidade Econômica e Ecossistema de Inovação  
**Foco:** Joinville/SC  
**Autor:** José Pedro  
**Data:** 2024

Este notebook fornece uma análise exploratória interativa dos dados econômicos utilizando o pipeline EcoMap.BR.

## Objetivos
1. Carregar e explorar dados de múltiplas fontes (RAIS, CAGED, PIB, ComexStat, DataViva)
2. Calcular indicadores econômicos (LQ, RCA, HHI)
3. Analisar séries temporais e sazonalidade
4. Criar visualizações interativas
5. Identificar padrões e insights

## 1. Setup Inicial e Importações

In [ ]:
# Imports necessários
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import yaml
from pathlib import Path
import sys

# Adiciona o diretório src ao path
sys.path.append('../src')
sys.path.append('..')

# Imports dos módulos locais
from src.utils.io_utils import DataLoader
from src.utils.data_cleaning import DataCleaner
from src.utils.validation import DataValidator, QualityChecker
from src.ingestion import EcoMapIngester
from src.indicators import EconomicIndicators, calculate_all_indicators
from src.visualization import EcoMapVisualizer, create_all_visualizations

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("✓ Imports realizados com sucesso!")

In [ ]:
# Carrega configurações
config_path = "../config/config.yaml"

try:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print("✓ Configurações carregadas")
    print(f"Região de foco: {config['geographic']['target_municipality']}")
    print(f"Período: {config['temporal']['start_year']} - {config['temporal']['end_year']}")
except Exception as e:
    print(f"❌ Erro ao carregar configurações: {e}")
    # Configurações padrão
    config = {
        'geographic': {'target_municipality': 'Joinville'},
        'temporal': {'start_year': 2018, 'end_year': 2023},
        'performance': {'preferred_engine': 'pandas'}
    }

## 2. Carregamento e Exploração de Dados

In [ ]:
# Inicializa o ingestor
ingester = EcoMapIngester(config)
print("✓ Ingestor inicializado")

# Verifica diretórios de dados
data_paths = {
    'RAIS': Path('../Coletas/RAIS'),
    'CAGED': Path('../Coletas/CAGED'),
    'ComexStat': Path('../Coletas/ComexStat'),
    'DataViva': Path('../Coletas/DataViva')
}

print("\nDisponibilidade de dados:")
for source, path in data_paths.items():
    if path.exists():
        files = list(path.glob('*.*'))
        print(f"✓ {source}: {len(files)} arquivo(s)")
    else:
        print(f"❌ {source}: diretório não encontrado")

In [ ]:
# Carrega dados disponíveis
print("Carregando dados...")
data_dict = {}

# RAIS
try:
    rais_data = ingester.ingest_rais_data()
    if rais_data is not None and len(rais_data) > 0:
        data_dict['rais'] = rais_data
        print(f"✓ RAIS: {len(rais_data)} registros")
    else:
        print("⚠ RAIS: sem dados")
except Exception as e:
    print(f"❌ RAIS: {str(e)[:100]}...")

# CAGED
try:
    caged_data = ingester.ingest_caged_data()
    if caged_data is not None and len(caged_data) > 0:
        data_dict['caged'] = caged_data
        print(f"✓ CAGED: {len(caged_data)} registros")
    else:
        print("⚠ CAGED: sem dados")
except Exception as e:
    print(f"❌ CAGED: {str(e)[:100]}...")

# ComexStat
try:
    comex_data = ingester.ingest_comexstat_data()
    if comex_data is not None and len(comex_data) > 0:
        data_dict['comexstat'] = comex_data
        print(f"✓ ComexStat: {len(comex_data)} registros")
    else:
        print("⚠ ComexStat: sem dados")
except Exception as e:
    print(f"❌ ComexStat: {str(e)[:100]}...")

# DataViva
try:
    dataviva_data = ingester.ingest_dataviva_data()
    if dataviva_data is not None and len(dataviva_data) > 0:
        data_dict['dataviva'] = dataviva_data
        print(f"✓ DataViva: {len(dataviva_data)} registros")
    else:
        print("⚠ DataViva: sem dados")
except Exception as e:
    print(f"❌ DataViva: {str(e)[:100]}...")

print(f"\nTotal de fontes carregadas: {len(data_dict)}")

In [ ]:
# Explora estrutura dos dados carregados
for source_name, df in data_dict.items():
    print(f"\n=== {source_name.upper()} ===")
    print(f"Shape: {df.shape}")
    print(f"Colunas: {list(df.columns)[:10]}{'...' if len(df.columns) > 10 else ''}")
    
    # Converte para pandas se necessário para análise
    if hasattr(df, 'to_pandas'):
        df_pd = df.to_pandas()
    else:
        df_pd = df
    
    # Informações básicas
    print(f"Tipos de dados: {df_pd.dtypes.value_counts().to_dict()}")
    print(f"Valores nulos: {df_pd.isnull().sum().sum()}")
    
    # Amostra dos dados
    print("\nAmostra:")
    display(df_pd.head(3))

## 3. Análise Descritiva

In [ ]:
# Estatísticas descritivas por fonte
print("=== ESTATÍSTICAS DESCRITIVAS ===")

for source_name, df in data_dict.items():
    print(f"\n--- {source_name.upper()} ---")
    
    # Converte para pandas para análise
    if hasattr(df, 'to_pandas'):
        df_pd = df.to_pandas()
    else:
        df_pd = df
    
    # Colunas numéricas
    numeric_cols = df_pd.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("Colunas numéricas:")
        display(df_pd[numeric_cols].describe())
    
    # Colunas categóricas (amostra)
    categorical_cols = df_pd.select_dtypes(include=['object']).columns[:5]
    if len(categorical_cols) > 0:
        print("\nPrincipais categorias:")
        for col in categorical_cols:
            print(f"{col}: {df_pd[col].nunique()} valores únicos")
            if df_pd[col].nunique() <= 20:
                print(f"  Top 5: {df_pd[col].value_counts().head().to_dict()}")

In [ ]:
# Análise temporal (se dados de tempo disponíveis)
print("=== ANÁLISE TEMPORAL ===")

for source_name, df in data_dict.items():
    if hasattr(df, 'to_pandas'):
        df_pd = df.to_pandas()
    else:
        df_pd = df
    
    # Procura colunas de tempo
    time_cols = [col for col in df_pd.columns if any(keyword in col.lower() 
                                                    for keyword in ['ano', 'year', 'data', 'date', 'mes', 'month'])]
    
    if time_cols:
        print(f"\n{source_name.upper()} - Cobertura temporal:")
        for col in time_cols[:2]:  # Limita a 2 colunas
            print(f"  {col}: {df_pd[col].min()} até {df_pd[col].max()}")
            print(f"  Valores únicos: {df_pd[col].nunique()}")

## 4. Cálculo de Indicadores Econômicos

In [ ]:
# Calcula indicadores usando o módulo
print("=== CALCULANDO INDICADORES ECONÔMICOS ===")

try:
    indicators_dict = calculate_all_indicators(data_dict, config)
    print(f"✓ {len(indicators_dict)} indicadores calculados")
    
    # Lista indicadores disponíveis
    for indicator_name, result in indicators_dict.items():
        if result is not None:
            if hasattr(result, 'shape'):
                print(f"  {indicator_name}: {result.shape}")
            else:
                print(f"  {indicator_name}: {type(result)}")
        else:
            print(f"  {indicator_name}: None")

except Exception as e:
    print(f"❌ Erro no cálculo de indicadores: {e}")
    indicators_dict = {}

In [ ]:
# Análise do Location Quotient (se disponível)
if 'location_quotient' in indicators_dict and indicators_dict['location_quotient'] is not None:
    lq_data = indicators_dict['location_quotient']
    
    # Converte para pandas se necessário
    if hasattr(lq_data, 'to_pandas'):
        lq_df = lq_data.to_pandas()
    else:
        lq_df = lq_data
    
    print("=== LOCATION QUOTIENT (LQ) ===")
    print(f"Shape: {lq_df.shape}")
    print("\nEstatísticas do LQ:")
    display(lq_df['location_quotient'].describe())
    
    # Setores com maior especialização (LQ > 1.2)
    high_lq = lq_df[lq_df['location_quotient'] > 1.2].sort_values('location_quotient', ascending=False)
    if len(high_lq) > 0:
        print(f"\nSetores especializados (LQ > 1.2): {len(high_lq)}")
        display(high_lq[['municipio', 'cnae', 'location_quotient', 'specialization']].head(10))
    
    # Joinville específico
    target_city = config['geographic']['target_municipality']
    joinville_lq = lq_df[lq_df['municipio'].str.contains(target_city, case=False, na=False)]
    if len(joinville_lq) > 0:
        print(f"\n{target_city} - Top 10 setores por LQ:")
        display(joinville_lq.nlargest(10, 'location_quotient')[['cnae', 'location_quotient', 'specialization']])

In [ ]:
# Análise do HHI (Concentração)
if 'hhi_concentration' in indicators_dict and indicators_dict['hhi_concentration'] is not None:
    hhi_data = indicators_dict['hhi_concentration']
    
    # Converte para pandas se necessário
    if hasattr(hhi_data, 'to_pandas'):
        hhi_df = hhi_data.to_pandas()
    else:
        hhi_df = hhi_data
    
    print("=== ÍNDICE DE CONCENTRAÇÃO (HHI) ===")
    print(f"Shape: {hhi_df.shape}")
    print("\nEstatísticas do HHI:")
    display(hhi_df['hhi'].describe())
    
    print("\nDistribuição por nível de concentração:")
    display(hhi_df['concentration_level'].value_counts())
    
    # Joinville específico
    target_city = config['geographic']['target_municipality']
    joinville_hhi = hhi_df[hhi_df['municipio'].str.contains(target_city, case=False, na=False)]
    if len(joinville_hhi) > 0:
        print(f"\n{target_city}:")
        display(joinville_hhi[['municipio', 'hhi', 'concentration_level', 'num_sectors']].head())

## 5. Visualizações Interativas

In [ ]:
# Inicializa visualizador
visualizer = EcoMapVisualizer(config)
print("✓ Visualizador inicializado")

In [ ]:
# Dashboard de indicadores
if len(indicators_dict) > 0:
    try:
        dashboard = visualizer.plot_indicators_dashboard(
            indicators_dict, 
            target_region=config['geographic']['target_municipality']
        )
        dashboard.show()
        print("✓ Dashboard de indicadores exibido")
    except Exception as e:
        print(f"❌ Erro no dashboard: {e}")
else:
    print("⚠ Sem indicadores para dashboard")

In [ ]:
# Visualização de Location Quotient (Treemap)
if 'location_quotient' in indicators_dict and indicators_dict['location_quotient'] is not None:
    try:
        lq_data = indicators_dict['location_quotient']
        
        # Filtra para Joinville
        target_city = config['geographic']['target_municipality']
        if hasattr(lq_data, 'to_pandas'):
            lq_df = lq_data.to_pandas()
        else:
            lq_df = lq_data
        
        lq_joinville = lq_df[lq_df['municipio'].str.contains(target_city, case=False, na=False)]
        
        if len(lq_joinville) > 0:
            treemap_fig = visualizer.plot_treemap(
                lq_joinville.head(20),  # Top 20 setores
                values_col='emp_regional_sector',
                names_col='cnae',
                color_col='location_quotient',
                title=f'Location Quotient por Setor - {target_city}'
            )
            treemap_fig.show()
            print("✓ Treemap do LQ exibido")
        else:
            print(f"⚠ Sem dados de LQ para {target_city}")
            
    except Exception as e:
        print(f"❌ Erro no treemap: {e}")

In [ ]:
# Análise temporal simples
for source_name, df in data_dict.items():
    try:
        # Converte para pandas
        if hasattr(df, 'to_pandas'):
            df_pd = df.to_pandas()
        else:
            df_pd = df
        
        # Procura colunas de ano e valor
        year_cols = [col for col in df_pd.columns if 'ano' in col.lower() or 'year' in col.lower()]
        value_cols = [col for col in df_pd.columns if any(keyword in col.lower() 
                     for keyword in ['emprego', 'job', 'saldo', 'valor', 'pib'])]
        
        if year_cols and value_cols:
            year_col = year_cols[0]
            value_col = value_cols[0]
            
            # Agrupa por ano
            yearly_data = df_pd.groupby(year_col)[value_col].sum().reset_index()
            
            if len(yearly_data) > 1:
                # Cria gráfico de linha
                fig = px.line(
                    yearly_data,
                    x=year_col,
                    y=value_col,
                    title=f'Evolução Temporal - {source_name.upper()}'
                )
                fig.show()
                print(f"✓ Série temporal {source_name} exibida")
                
    except Exception as e:
        print(f"❌ Erro na série temporal {source_name}: {e}")

## 6. Análise de Correlações

In [ ]:
# Matriz de correlação (se tabela mestre disponível)
if 'master_table' in indicators_dict and indicators_dict['master_table'] is not None:
    try:
        master_table = indicators_dict['master_table']
        
        # Converte para pandas
        if hasattr(master_table, 'to_pandas'):
            master_df = master_table.to_pandas()
        else:
            master_df = master_table
        
        # Seleciona colunas numéricas
        numeric_cols = master_df.select_dtypes(include=[np.number]).columns[:15]  # Limita a 15 colunas
        
        if len(numeric_cols) > 3:
            corr_matrix = master_df[numeric_cols].corr()
            
            # Heatmap interativo
            fig = px.imshow(
                corr_matrix,
                text_auto=True,
                aspect="auto",
                title="Matriz de Correlação entre Indicadores",
                color_continuous_scale='RdBu_r'
            )
            fig.show()
            print("✓ Matriz de correlação exibida")
            
    except Exception as e:
        print(f"❌ Erro na matriz de correlação: {e}")
else:
    print("⚠ Tabela mestre não disponível para correlações")

## 7. Insights e Resumo

In [ ]:
# Resumo dos principais insights
print("=== RESUMO DOS PRINCIPAIS INSIGHTS ===")

target_city = config['geographic']['target_municipality']
print(f"\nAnálise para: {target_city}")
print(f"Período: {config['temporal']['start_year']} - {config['temporal']['end_year']}")
print(f"Fontes de dados analisadas: {len(data_dict)}")
print(f"Indicadores calculados: {len([k for k, v in indicators_dict.items() if v is not None])}")

# Location Quotient insights
if 'location_quotient' in indicators_dict and indicators_dict['location_quotient'] is not None:
    lq_data = indicators_dict['location_quotient']
    if hasattr(lq_data, 'to_pandas'):
        lq_df = lq_data.to_pandas()
    else:
        lq_df = lq_data
    
    joinville_lq = lq_df[lq_df['municipio'].str.contains(target_city, case=False, na=False)]
    if len(joinville_lq) > 0:
        specialized_sectors = joinville_lq[joinville_lq['location_quotient'] > 1.2]
        print(f"\n📊 Location Quotient:")
        print(f"  - Setores especializados: {len(specialized_sectors)}")
        print(f"  - LQ médio: {joinville_lq['location_quotient'].mean():.2f}")
        if len(specialized_sectors) > 0:
            top_sector = specialized_sectors.nlargest(1, 'location_quotient').iloc[0]
            print(f"  - Setor mais especializado: {top_sector['cnae']} (LQ: {top_sector['location_quotient']:.2f})")

# HHI insights
if 'hhi_concentration' in indicators_dict and indicators_dict['hhi_concentration'] is not None:
    hhi_data = indicators_dict['hhi_concentration']
    if hasattr(hhi_data, 'to_pandas'):
        hhi_df = hhi_data.to_pandas()
    else:
        hhi_df = hhi_data
    
    joinville_hhi = hhi_df[hhi_df['municipio'].str.contains(target_city, case=False, na=False)]
    if len(joinville_hhi) > 0:
        hhi_value = joinville_hhi['hhi'].iloc[0]
        concentration_level = joinville_hhi['concentration_level'].iloc[0]
        print(f"\n🎯 Concentração Setorial (HHI):")
        print(f"  - Índice HHI: {hhi_value:.3f}")
        print(f"  - Nível de concentração: {concentration_level}")
        print(f"  - Número de setores: {joinville_hhi['num_sectors'].iloc[0]}")

# Crescimento insights
growth_keys = [k for k in indicators_dict.keys() if 'growth' in k and indicators_dict[k] is not None]
if growth_keys:
    print(f"\n📈 Análise de Crescimento:")
    print(f"  - Séries temporais analisadas: {len(growth_keys)}")
    for growth_key in growth_keys[:3]:  # Limita a 3
        try:
            growth_data = indicators_dict[growth_key]
            if hasattr(growth_data, 'to_pandas'):
                growth_df = growth_data.to_pandas()
            else:
                growth_df = growth_data
            
            joinville_growth = growth_df[growth_df['municipio'].str.contains(target_city, case=False, na=False)]
            if len(joinville_growth) > 0:
                growth_cols = [col for col in growth_df.columns if 'growth' in col]
                if growth_cols:
                    avg_growth = joinville_growth[growth_cols[0]].mean()
                    print(f"  - {growth_key}: crescimento médio {avg_growth:.1%}")
        except Exception as e:
            pass

print("\n" + "="*50)
print("Análise exploratória concluída!")
print("Para análises mais detalhadas, utilize os módulos específicos do EcoMap.BR")
print("="*50)

## 8. Próximos Passos

### Análises Recomendadas:
1. **Análise Setorial Detalhada**: Aprofundar nos setores com maior LQ
2. **Comparação Regional**: Comparar Joinville com outras cidades similares
3. **Análise de Exportações**: Identificar produtos com RCA > 1
4. **Sazonalidade**: Analisar padrões mensais no emprego
5. **Projeções**: Modelar tendências futuras

### Comandos CLI:
```bash
# Pipeline completo
python main.py all

# Apenas visualizações
python main.py viz

# Relatório automatizado
python main.py report
```

### Configurações:
- Ajuste `config/config.yaml` para diferentes regiões ou períodos
- Modifique filtros geográficos e temporais
- Habilite/desabilite indicadores específicos

### Saídas:
- Dados processados: `outputs/processed_data/`
- Indicadores: `outputs/indicators/`
- Visualizações: `outputs/visualizations/`
- Relatórios: `outputs/reports/`